In [7]:
%pip install --upgrade pip
%pip install --upgrade transformers datasets[audio] accelerate

Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset  1번


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True, # 2번
    chunk_length_s = 10, # 3번
    stride_length_s= 2, # 3번
)

#dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
#sample = dataset[0]["audio"] # 1번
sample = "./audio/회의01.mp3"

result = pipe(sample)
#print(result["text"]) # 4번

print(result)


Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


{'text': ' 어떻게 돼서 이게 안 붙은 상태에서 붙은 거 이해 가시죠. 여기서 시작을 하는 건데 걔네도 여기서 뭔가 시작할 방법이 없으니까 스케치업에서 내가 면을 만든 거잖아요. 그 과정까지는 이해는 하는데 현재 취업에서 내가 면을 만든 거잖아요 그 과정까지는 이해는 하는데 그거를 안고치고 트레드파에서 어떻게 바로 면을 바로 만든다거나 이런 방법이 있을지 한번 한번 고민만 좀 해보세요 파일은 제가 드리고 갈텐데 아마 힘들 거에서 아무튼 혹시라도 아이디어가 있나 해서 여쭤보는 거니까 알겠습니다. 테이프 해보자. 네 됐습니다. 알겠습니다.', 'chunks': [{'timestamp': (0.0, 5.0), 'text': ' 어떻게 돼서 이게 안 붙은 상태에서 붙은 거 이해 가시죠.'}, {'timestamp': (5.0, 10.0), 'text': ' 여기서 시작을 하는 건데 걔네도 여기서 뭔가 시작할 방법이 없으니까'}, {'timestamp': (10.0, 13.0), 'text': ' 스케치업에서 내가 면을 만든 거잖아요.'}, {'timestamp': (13.0, 14.0), 'text': ' 그 과정까지는 이해는 하는데 현재 취업에서 내가 면을 만든 거잖아요'}, {'timestamp': (14.0, 16.0), 'text': ' 그 과정까지는 이해는 하는데'}, {'timestamp': (16.0, 18.0), 'text': ' 그거를'}, {'timestamp': (18.0, 20.0), 'text': ' 안고치고 트레드파에서'}, {'timestamp': (20.0, 22.0), 'text': ' 어떻게 바로'}, {'timestamp': (22.0, 24.0), 'text': ' 면을 바로 만든다거나'}, {'timestamp': (24.0, 26.0), 'text': ' 이런 방법이 있을지 한번'}, {'timestamp': (26.0, 28.0), 'text': ' 한번 고민만 좀 해보세요'}, {'timestamp': (28.0, 3

In [9]:
#chunks 를 CSV파일로 저장
start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("lsy_회의01.csv", index=False, sep="|")
display(df)

,start,end,text
0,0.0,5.0,어떻게 돼서 이게 안 붙은 상태에서 붙은 거 이해 가시죠.
1,5.0,10.0,여기서 시작을 하는 건데 걔네도 여기서 뭔가 시작할 방법이 없으니까
2,10.0,13.0,스케치업에서 내가 면을 만든 거잖아요.
3,13.0,14.0,그 과정까지는 이해는 하는데 현재 취업에서 내가 면을 만든 거잖아요
4,14.0,16.0,그 과정까지는 이해는 하는데
5,16.0,18.0,그거를
6,18.0,20.0,안고치고 트레드파에서
7,20.0,22.0,어떻게 바로
8,22.0,24.0,면을 바로 만든다거나
9,24.0,26.0,이런 방법이 있을지 한번
